In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data 
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
data = fetch_olivetti_faces()
X = data.images
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [3]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

In [4]:
X_train.shape

(280, 4096)

In [5]:
X_test.shape

(120, 4096)

In [6]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/olivetti.json")  
gc = GCForest(config)

In [7]:
# count class
len(np.unique(y_train))

40

In [8]:
     # X_enc is the concatenated predict_proba result of ExtraTrees
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
     

[ 2018-04-23 09:00:10,476][cascade_classifier.fit_transform] X_groups_train.shape=[(280, 4096)],y_train.shape=(280,),X_groups_test.shape=[(120, 4096)],y_test.shape=(120,)
[ 2018-04-23 09:00:10,481][cascade_classifier.fit_transform] group_dims=[4096]
[ 2018-04-23 09:00:10,483][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-23 09:00:10,484][cascade_classifier.fit_transform] group_ends=[4096]
[ 2018-04-23 09:00:10,486][cascade_classifier.fit_transform] X_train.shape=(280, 4096),X_test.shape=(120, 4096)
[ 2018-04-23 09:00:10,492][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4096), X_cur_test.shape=(120, 4096)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[ 2018-04-23 09:00:11,193][kfold_wrapper.log_e

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees  = {:.6f} %".format(acc * 100))

[ 2018-04-23 09:00:44,785][cascade_classifier.transform] X_groups_test.shape=[(120, 4096)]
[ 2018-04-23 09:00:44,790][cascade_classifier.transform] group_dims=[4096]
[ 2018-04-23 09:00:44,792][cascade_classifier.transform] X_test.shape=(120, 4096)
[ 2018-04-23 09:00:44,794][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(120, 4096)


Test Accuracy of ExtratTrees  = 96.666667 %


In [10]:
#  predict_proba result of ExtraTrees + data
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-23 09:00:46,339][cascade_classifier.transform] X_groups_test.shape=[(120, 4096)]
[ 2018-04-23 09:00:46,343][cascade_classifier.transform] group_dims=[4096]
[ 2018-04-23 09:00:46,344][cascade_classifier.transform] X_test.shape=(120, 4096)
[ 2018-04-23 09:00:46,346][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(120, 4096)


In [11]:
X_test_enc.shape

(120, 4136)

In [12]:
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [13]:
X_train_new.shape

(280, 4136)

In [14]:
# casacade ExtraTrees
config = load_json("./examples/olivettigc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-23 09:00:48,909][cascade_classifier.fit_transform] X_groups_train.shape=[(280, 4136)],y_train.shape=(280,),X_groups_test.shape=[(120, 4136)],y_test.shape=(120,)
[ 2018-04-23 09:00:48,913][cascade_classifier.fit_transform] group_dims=[4136]
[ 2018-04-23 09:00:48,914][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-23 09:00:48,916][cascade_classifier.fit_transform] group_ends=[4136]
[ 2018-04-23 09:00:48,918][cascade_classifier.fit_transform] X_train.shape=(280, 4136),X_test.shape=(120, 4136)
[ 2018-04-23 09:00:48,925][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4136), X_cur_test.shape=(120, 4136)
[ 2018-04-23 09:00:49,539][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-23 09:00:50,300][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=86.67%
[ 2018-04-23 09:00:51,080][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - esti

In [15]:
def update(X_train,X_test):
    # update Estimators
    for i in range(20):
        M = np.delete(X_train, (-1), axis=1)
        X_train= np.delete(M, (-1), axis=1)
        M = np.delete(X_test, (-1), axis=1)
        X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')
    

In [16]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/olivettigc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-23 09:01:22,080][cascade_classifier.fit_transform] X_groups_train.shape=[(280, 4136)],y_train.shape=(280,),X_groups_test.shape=[(120, 4136)],y_test.shape=(120,)
[ 2018-04-23 09:01:22,082][cascade_classifier.fit_transform] group_dims=[4136]
[ 2018-04-23 09:01:22,084][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-23 09:01:22,085][cascade_classifier.fit_transform] group_ends=[4136]
[ 2018-04-23 09:01:22,087][cascade_classifier.fit_transform] X_train.shape=(280, 4136),X_test.shape=(120, 4136)
[ 2018-04-23 09:01:22,089][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4136), X_cur_test.shape=(120, 4136)
[ 2018-04-23 09:01:22,667][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.00%
[ 2018-04-23 09:01:23,440][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=80.00%
[ 2018-04-23 09:01:24,250][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

[ 2018-04-23 09:01:56,156][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=92.50%
[ 2018-04-23 09:01:56,967][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=85.00%
[ 2018-04-23 09:01:57,809][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=92.11%
[ 2018-04-23 09:01:58,662][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=92.86%
[ 2018-04-23 09:01:59,489][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=95.00%
[ 2018-04-23 09:02:00,316][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-23 09:02:01,106][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=86.67%
[ 2018-04-23 09:02:01,920][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=100.00

('Test Accuracy of Extratrees = 97.500000 %', 0, 'fois')


[ 2018-04-23 09:02:06,736][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=92.50%
[ 2018-04-23 09:02:07,622][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=86.67%
[ 2018-04-23 09:02:08,395][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=92.86%
[ 2018-04-23 09:02:09,204][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=88.57%
[ 2018-04-23 09:02:09,996][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-23 09:02:10,840][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=89.47%
[ 2018-04-23 09:02:11,614][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=95.00%
[ 2018-04-23 09:02:12,458][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=87.50%

[ 2018-04-23 09:02:44,654][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-23 09:02:45,479][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=86.67%
[ 2018-04-23 09:02:46,366][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=100.00%
[ 2018-04-23 09:02:47,208][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=88.57%
[ 2018-04-23 09:02:47,361][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=91.43%
[ 2018-04-23 09:02:47,362][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=96.67%
[ 2018-04-23 09:02:47,364][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=91.43%
[ 2018-04-23 09:02:47,366][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=96.67%
[ 2018-04-23 09:02:47,373

...

[ 2018-04-23 09:17:06,602][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-23 09:17:07,447][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=86.67%
[ 2018-04-23 09:17:08,240][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=100.00%
[ 2018-04-23 09:17:09,088][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=91.43%
[ 2018-04-23 09:17:09,238][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=91.43%
[ 2018-04-23 09:17:09,240][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.50%
[ 2018-04-23 09:17:09,242][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=91.43%
[ 2018-04-23 09:17:09,244][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.50%
[ 2018-04-23 09:17:09,252

('Test Accuracy of Extratrees = 97.500000 %', 18, 'fois')


[ 2018-04-23 09:17:22,899][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=92.50%
[ 2018-04-23 09:17:23,671][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=86.67%
[ 2018-04-23 09:17:24,476][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=89.29%
[ 2018-04-23 09:17:25,268][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=88.57%
[ 2018-04-23 09:17:26,175][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-23 09:17:26,975][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=92.11%
[ 2018-04-23 09:17:27,799][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=95.00%
[ 2018-04-23 09:17:28,636][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=82.50%

[ 2018-04-23 09:18:00,680][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-23 09:18:01,540][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=86.67%
[ 2018-04-23 09:18:02,335][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=100.00%
[ 2018-04-23 09:18:03,172][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=94.29%
[ 2018-04-23 09:18:03,323][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=92.14%
[ 2018-04-23 09:18:03,325][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=96.67%
[ 2018-04-23 09:18:03,327][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=92.14%
[ 2018-04-23 09:18:03,329][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=96.67%
[ 2018-04-23 09:18:03,404

('Test Accuracy of Extratrees = 96.666667 %', 19, 'fois')


# ('Test Accuracy of Extratrees = 98.333333 %', 2, 'fois')